najpierw instalujemy srodowisko

In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_bh0dwbs
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_bh0dwbs
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=c151cb95cea0b13c4f7ec17546bb319d29f0034212a98372edfefae2a7216528
  Stored in directory: /tmp/pip-ephem-wheel-cache-f4kcrjcz/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
cr

**metoda prostokątów na CPU**<br>
xp <- początek przedziału<br>
xk <- koniec przedziału<br>
n <- ustawiamy ilosc "prostokątów" <br>
w f1 możemy zedytować i dodac funkcje jaką chcemy<br>

In [ ]:
%%cu
#include <iostream>
#include <cstdlib>

using namespace std;

// funkcja do scalkowania
double f1(double x) { return -x*x-x+20; }

int main()
{
    float xp, xk, h, calka;
    int n;

    // przedzialy
    xp = -2;
    xk = 1;

    // im wieksze n tym wieksza dokladnosc (tym wiecej prostokątow)
    n = 12;

    h = (xk - xp) / (float)n;

    cout << "krok: h=" << h << endl;

    calka = 0;

    for (int i=1; i<=n; i++)
    {
        calka += f1(xp + i*h)*h;
    }

    cout << "Wynik calkowania metoda prostokatow: " <<  calka << endl;

    system("PAUSE");
    return 0;
}  

krok: h=0.25
Wynik calkowania metoda prostokatow: 58.4688
sh: 1: PAUSE: not found



**metoda prostokątów CPU**<br>
xp <- początek przedziału<br>
xk <- koniec przedziału<br>
n <- ustawiamy ilosc "prostokątów" <br>
w f1 możemy zedytować i dodac funkcje jaką chcemy<br>

**opis metody prostokątów**<br>
Metoda trapezów daje dość dokładne wyniki, szczególnie przy dużej dokładności N. Polega ona na podzieleniu obszaru całowanego na x trapezów (przedziałów), których pola na końcu sumuje się.

In [ ]:
%%cu
#include <iostream>
#include <cstdlib>

using namespace std;

// funkcja do scalkowania
double f1(double x) { return -x*x-x+20; }

int main()
{
    float xp, xk, h, calka;
    int n;

    // przedzialy
    xp = -2;
    xk = 1;

    // im wieksze n tym wieksza dokladnośc (np. n=1000)
    n = 10;

    h = (xk - xp) / (float)n;

    cout << "krok: h=" << h << endl;

    calka = 0;
    for (int i=1; i<n; i++)
    {
        calka += f1(xp + i * h);
    }
    calka += f1(xp) / 2;
    calka += f1(xk) / 2;
    calka *= h;

    cout << "Wynik calkowania: " <<  calka << endl;

    system("PAUSE");

    return 0;
}

krok: h=0.3
Wynik calkowania: 58.455
sh: 1: PAUSE: not found



** metoda simpsona CPU**<br>
xp <- początek przedziału<br>
xk <- koniec przedziału<br>
n <- ustawiamy ilosc "prostokątów" <br>
w f1 możemy zedytować i dodac funkcje jaką chcemy<br>

In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
float f1(float x) {
 return -x*x-x+20;
}
 
//get data, algorithm
int main(){
float xp, xk, dx, calka, s, x;
int i, n;
 
    // przedzialy
    xp = -2;
    xk = 1;

    // im wieksze n tym wieksza dokladnośc (np. n=1000)
    n = 3;
dx = (xk - xp) / (float)n;
 
calka = 0;
s = 0;
for (i=1; i<n; i++) {
x = xp + i*dx;
s += f1(x - dx / 2);
calka += f1(x);
}
s += f1(xk - dx / 2);
calka = (dx/6) * (f1(xp) + f1(xk) + 2*calka + 4*s);
 
 
printf("Wartosc calki wynosi w przyblizeniu %f\n", calka);
 
return 0;
}

Wartosc calki wynosi w przyblizeniu 58.500000



**metoda simpsona gpu**

In [ ]:
%%cu

#include <iostream>


// tutaj dzielimy problem na mniejsze problemy
    __device__ void initThreadBounds(int *n_start, int *n_end, int n, 
                                        int totalBlocks, int blockWidth)
    {
        int threadId = blockWidth * blockIdx.x + threadIdx.x;
        int nextThreadId = threadId + 1;

        int threads = blockWidth * totalBlocks;

        *n_start = (threadId * n)/ threads;
        *n_end =  (nextThreadId * n)/ threads;
    }

// nasz funkacja
    __device__ float reg_func (float x)
    {
        return -x*x-x+20;
    }

    typedef float (*p_func) (float);

    __device__ p_func integrale_f = reg_func;


// tutaj liczymy calke metoda simpsona
    __device__ void integralSimpsonMethod(int totalBlocks, int totalThreads, 
                    double a, double b, int n, float p_function(float), float* result)
    {
        *result = 0;

        float h = (b - a)/n; 
        //*result = p_function(a)+p_function(a + h * n);
        //parallel
        int idx_start;
        int idx_end;
        initThreadBounds(&idx_start, &idx_end, n-1, totalBlocks, totalThreads);
        //parallel_ends
        for (int i = idx_start; i < idx_end; i+=2) {
            *result +=  ( p_function(a + h*(i-1)) + 
                          4 * p_function(a + h*(i)) + 
                          p_function(a + h*(i+1)) ) * h/3;

        }   
    } 


    __global__ void integralSimpson(int totalBlocks, int totalThreads,  float* result)
    {
        float res = 0;

        integralSimpsonMethod(totalBlocks, totalThreads, 0, 10, 90, integrale_f, &res);
        result[(blockIdx.x*totalThreads + threadIdx.x)] = res;

        //printf ("Simpson method\n");
    }

//zbieramy wszystko w calosc
    __host__ void inttest()
    {

        const int blocksNum = 9;
        const int threadNum = 9;

        float   *device_resultf; 
        float   host_resultf[threadNum*blocksNum]={0};


        cudaMalloc((void**) &device_resultf, sizeof(float)*threadNum*blocksNum);
            integralSimpson<<<blocksNum, threadNum>>>(blocksNum, threadNum, device_resultf);
        cudaThreadSynchronize();

        cudaMemcpy(host_resultf, device_resultf, sizeof(float) *threadNum*blocksNum, 
                      cudaMemcpyDeviceToHost);

        float sum = 0;
        for (int i = 0; i != blocksNum*threadNum; ++i) {
            sum += host_resultf[i];
            //  printf ("result in %i cell = %f \n", i, host_resultf[i]);
        }
        printf ("sum = %f \n", sum);
        cudaFree(device_resultf);
    }

int main(int argc, char* argv[])
{


   inttest();


    int i = 3;

}

**metoda prostokatow gpu**

In [ ]:
%%cu
#include <stdio.h>

__global__ void integrateKernel(double a, double b, int n, double *sum) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  double dx = (b - a) / n;
  if (tid < n) {
    double x = a + dx * tid;
    double funcVal =-x*x-x+20; // nasza funkcja
    sum[tid] = funcVal * dx;
  }
  __syncthreads();
}

int main() {
  double a = 0.0; // Lower limit of integration
  double b = 2.0; // Upper limit of integration
  int n = 100; // Number of intervals

  // alokujemy pamiec
  double *h_sum = new double[n];
  double *d_sum;
  cudaMalloc(&d_sum, n * sizeof(double));
  cudaMemcpy(d_sum, h_sum, n * sizeof(double), cudaMemcpyHostToDevice);

  // tutaj uruchamiamy metode
  integrateKernel<<<1, n>>>(a, b, n, d_sum);
  
  // pamiec z GPU do CPU
  cudaMemcpy(h_sum, d_sum, n * sizeof(double), cudaMemcpyDeviceToHost);

  // sumujemy
  double sum = 0.0;
  for (int i = 0; i < n; i++) {
    sum += h_sum[i];
  }
  
  printf("The integral is: %f\n", sum);

  cudaFree(d_sum);

  return 0;
}